## Subclassing Built-In Types Is Tricky
The code of the built-ins (written in C) does not call special methods overridden by user-defined classes. For example, an overridden `__getitem__()` in a subclass of dict will not be called by e.g. the built-in `get()` method.

In [6]:
class DoppelDict(dict):
    def __setitem__(self, key, value):
        super().__setitem__(key, [value] * 2)
        
# invoke `__init__`, ignore user-defined method
dd = DoppelDict(one = 1)
# invoke `__setitem__`, work as expected
dd['two'] = 2
# ignore user-defined method
dd.update(three=3)

dd

{'one': 1, 'two': [2, 2], 'three': 3}

But if you subclass `collections.UserDict` instead of `dict`, these issues will be fixed.

In [7]:
import collections

class DoppelDict2(collections.UserDict):
    def __setitem__(self, key, value):
        super().__setitem__(key, [value] * 2)

dd = DoppelDict2(one = 1)
dd['two'] = 2
dd.update(three=3)
dd

{'one': [1, 1], 'two': [2, 2], 'three': [3, 3]}

## Multiple inheritance and Method Resolution Order
Diamond Problem: Any language implementing multiple inheritance needs to deal with potential naming conflicts when unrelated ancestor classes implement a method by the same name.

In [10]:
class A:
    def ping(self):
        print('ping: ', self)
class B(A):
    def pong(self):
        print('pong:', self)
class C(A):
    def pong(self):
        print('PONG:', self)

class D(B, C):
    def ping(self):
        super().ping()
        print('post-ping:', self)
    def pingpong(self):
        self.ping()
        super().ping()
        self.pong()
        super().pong()
        C.ping(self)

d = D()
d.pong()    # `pong` of B version
C.pong(d)   # `pong` of C version

pong: <__main__.D object at 0x7facd2218310>
PONG: <__main__.D object at 0x7facd2218310>


In [3]:
import torch
a = torch.scalar_tensor(0)
type(torch.stack([a,a,a]))

torch.Tensor